<a href="https://colab.research.google.com/github/ncukaran18csu103/karan_18csu103/blob/master/recomende__system_21_11_20220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec_sys').getOrCreate()
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [ ]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [ ]:
data.count()

1501

In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [ ]:
print((data['rating']==5))

Column<b'(rating = 5)'>


In [ ]:
# Build the recommendation model using ALS on the training data
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

als = ALS(maxIter=2, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   3.0|     8| -2.2357826|
|     65|   1.0|    28|  1.3509676|
|     65|   1.0|    19|  0.7031145|
|     65|   5.0|    23|  0.5500305|
|     65|   1.0|     2|  0.6697277|
|     53|   2.0|    19|  1.0796318|
|     53|   1.0|     9|     4.5861|
|     53|   5.0|     8|   2.699062|
|     53|   1.0|    23|  3.4064045|
|     53|   1.0|    25| -0.9313456|
|     78|   1.0|    28|  0.5947387|
|     78|   1.0|     1| 0.69995224|
|     78|   1.0|    13|  0.4997624|
|     78|   1.0|     4|-0.21823697|
|     34|   1.0|    16|  2.4193988|
|     34|   1.0|    15|  1.3011091|
|     81|   5.0|    28|  0.5506718|
|     81|   1.0|     6|  1.7837012|
|     28|   1.0|     5|  1.3949606|
|     28|   1.0|     2|  0.8820266|
+-------+------+------+-----------+
only showing top 20 rows



In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


evaluator = RegressionEvaluator(metricName="r2", labelCol="rating",predictionCol="prediction")
r2 = evaluator.evaluate(predictions)
print("r2= " + str(r2))


evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",predictionCol="prediction")
mae = evaluator.evaluate(predictions)
print("mean absolute error = " + str(mae))

Root-mean-square error = 1.9075057465953156
r2= -1.5413106774485312
mean absolute error = 1.4644186509977346


In [ ]:
single_user = test.filter(test['userId']==20).select(['movieId','userId','rating'])

In [ ]:
single_user.show()

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|     21|    20|   2.0|
|     29|    20|   2.0|
|     40|    20|   1.0|
|     50|    20|   1.0|
|     51|    20|   3.0|
|     64|    20|   1.0|
|     77|    20|   4.0|
|     90|    20|   3.0|
+-------+------+------+



In [ ]:
reccomendations = model.transform(single_user)

In [ ]:
reccomendations.sort('prediction',ascen=True).show()

+-------+------+------+------------+
|movieId|userId|rating|  prediction|
+-------+------+------+------------+
|     77|    20|   4.0|-0.026269102|
|     40|    20|   1.0|   0.2851828|
|      0|    20|   1.0|  0.53675526|
|     94|    20|   4.0|   1.0944935|
|     45|    20|   2.0|   1.1753477|
|     76|    20|   1.0|   1.2089039|
|     51|    20|   3.0|   1.7421172|
|     50|    20|   1.0|   1.8594538|
|     73|    20|   2.0|   1.8908287|
|     70|    20|   2.0|   1.9222051|
|     61|    20|   1.0|   1.9425522|
|     88|    20|   3.0|    1.955031|
|     39|    20|   1.0|    2.367053|
+-------+------+------+------------+

